Use the same method as CS_AIM when loading in images

First get all the image paths

In [ ]:
import tensorflow as tf

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

img_paths = []

for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        if(filename[-3:] == 'jpg' and 'train' in dirname):
            img_paths.append(os.path.join(dirname, filename))
        


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
IMG_WIDTH = 1024
IMG_HEIGHT = 1024

# UTILITIES
def load(image_file):
  # height, width, channels
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  input_image = tf.cast(image, tf.float32)

  return input_image

def resize(input_image, height, width):
  input_image = tf.image.resize(input_image, [height, width], 
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return input_image

def normalize(input_image):
  input_image = (input_image / 127.5) - 1
  return input_image

def load_image_train(image_file):

  input_image = load(image_file)
  input_image = resize(input_image, IMG_WIDTH, IMG_HEIGHT)
  input_image = normalize(input_image)

  return input_image

Create a dictionary mapping image names to labels, so that we can generate input and output tensors for training

In [ ]:
import pandas as pd
import os
df = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
#print(df.head())
#df.set_index('image_name', inplace=True, drop=True)


label_map = dict()
for r in range(len(df)):
    if r == 0:
        continue
    row = df.loc[r]
    label_map[row['image_name']] = row['target']# image name


Since we cannot load 105 GB of image data at once, and due to the data's structured in /input/, rather than using one of the built in functions to load data, we will create our own method to return a batch.

In [ ]:
import random
# Load in batch_size images, selected at random, and also get their labels
# Convert to tensor and return (batch_size, width, height, channels), (batch_size, # of outputs)
def get_batch(batch_size=32):
    img_1 = random.randint(0, len(img_paths))
    images = []
    labels = []
    
    for i in range(batch_size):
        img_path = random.randint(0, len(img_paths))
        images.append(load_image_train(img_paths[img_path]))
        labels.append(label_map[img_paths[img_1][-16:-4]])
        
        
    return tf.stack(images, axis=0), tf.convert_to_tensor(labels) # labels may not need to be a tensor
    
    # IMAGES: (BATCH_SIZE, width, height, channels)
    # LABEL: (BATCH_SIZE, )

In [ ]:
images, labels = get_batch()
labels.shape

In [ ]:
images.shape

For this challenge, I will implement densely connected convolutional networks combined with resnet

Implement a densely connected convolutional network with heavy use of dropout, batchnorm, etc. using a pretrained architecture

In [ ]:
inputs = tf.keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)) # batch size not included
x = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet')(inputs)
x = tf.keras.layers.SeparableConv2D(64, 3, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.SeparableConv2D(128, 2, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.SeparableConv2D(256, 1, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.SeparableConv2D(256, 4, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.MaxPooling2D()(x)
# Concat x3 and x2 to add dense connections
x3 = x
x = tf.keras.layers.SeparableConv2D(256, 2, strides=(1, 1), padding='same')(x)
x2 = x
x = tf.keras.layers.concatenate([x2, x3]) # Dense connection
x = tf.keras.layers.SeparableConv2D(256, 1, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024)(x)
model = tf.keras.Model(inputs=inputs, outputs=x)
model.output_shape

In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
model.get_layer('resnet50v2').trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
num_epochs = 50
batch_size = 16
steps = num_epochs // batch_size

for step in steps:
    x_train, y_train = get_batch(batch_size)
    history = model.fit(
        x_train,
        y_train,
        batch_size=16,
        epochs=1
    )